In [56]:
import os
import random
import json
import time
from io import BytesIO
import cv2
import base64
import logging
import uuid
import traitlets
import numpy as np
from gym_donkeycar.core.sim_client import SDClient


###########################################

class SimpleClient(SDClient, traitlets.HasTraits):
    image = traitlets.Any()
    imageB = traitlets.Any()
    throtle = traitlets.Float()
    stearing = traitlets.Float()
    
    def pipeline(self, src):
        zero = np.zeros([src.shape[0], src.shape[1]])
        blue = np.int16(np.matrix(src[:,:,0]))
        green = np.int16(np.matrix(src[:,:,1]))
        red = np.int16(np.matrix(src[:,:,2]))
        red_only = red-blue
        green_only = green-blue
        red_only[red_only<0] = 0
        red_only[red_only>255] = 255
        red_only = np.uint8(red_only)
        green_only[green_only<0] = 0
        green_only[green_only>255] = 255
        green_only = np.uint8(green_only)
        image =  green_only + red_only
        image[150:255,:] = 0
        contours, hierarchy = cv2.findContours(image ,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        c = max(contours, key=cv2.contourArea)

        M = cv2.moments(c)

        cx = int(M['m10']/M['m00'])

        cy = int(M['m01']/M['m00'])

        cv2.line(image,(cx,0),(cx,255),255,1)

        cv2.line(image,(0,cy),(255,cy),255,1)
        cv2.drawContours(image,[c],-1,255,3)
        self.stearing = 1 - (cx / 255.0)
        return image
    
    def __init__(self, address, racer, vehicle, poll_socket_sleep_time=0.01, *args, **kwargs):
        host, port = address
        traitlets.HasTraits.__init__(self, *args, **kwargs)
        SDClient.__init__(self, host, port, poll_socket_sleep_time=poll_socket_sleep_time)
        self.last_image = None
        self.car_loaded = False
        self.racer = racer
        self.vehicle = vehicle

    def on_msg_recv(self, json_packet):
        
        if json_packet['msg_type'] == "need_car_config":
            self.send_config(self.racer, self.vehicle)

        if json_packet['msg_type'] == "car_loaded":

            self.car_loaded = True
        
        if json_packet['msg_type'] == "telemetry":
            imgString = json_packet["image"]
            
            self.imageB = cv2.imdecode(np.frombuffer(base64.b64decode(imgString), dtype=np.uint8), cv2.IMREAD_COLOR)
            self.image = self.pipeline(self.imageB)

            #don't have to, but to clean up the print, delete the image string.
            del json_packet["image"]

            if "imageb" in json_packet:
                print('got image b')
                self.imageB = cv2.imdecode(np.frombuffer(base64.b64decode(imgString), dtype=np.uint8), cv2.IMREAD_COLOR)
                #don't have to, but to clean up the print, delete the image string.
                del json_packet["imageb"]

            if "lidar" in json_packet:
                lidar = json_packet["lidar"]
                if lidar is not None:
                    print("lidar:", len(lidar), "pts")

                #don't have to, but to clean up the print, delete the lidar string.
                del json_packet["lidar"]
                
        else:
            print(json_packet)


    def send_config(self, racer, vehicle):
        '''
        send three config messages to setup car, racer, and camera
        '''
        racer_name = racer
        car_name = vehicle
        bio = "I race robots."
        country = "France"
        guid = str(uuid.uuid4())

        # Racer info
        msg = {'msg_type': 'racer_info',
            'racer_name': racer_name,
            'car_name' : car_name,
            'bio' : bio,
            'country' : country,
            'guid' : guid }
        self.send_now(json.dumps(msg))
        time.sleep(0.2)
        
        # Car config
        # body_style = "donkey" | "bare" | "car01" choice of string
        # body_rgb  = (128, 128, 128) tuple of ints
        # car_name = "string less than 64 char"

        msg = '{ "msg_type" : "car_config", "body_style" : "donkey", "body_r" : "255", "body_g" : "0", "body_b" : "255", "car_name" : "%s", "font_size" : "100" }' % (car_name)
        self.send_now(msg)

        #this sleep gives the car time to spawn. Once it's spawned, it's ready for the camera config.
        time.sleep(0.2)
 
        msg = '''{
    "msg_type" : "cam_config",
    "fov" : "150", 
    "fish_eye_x" : "0.0",
    "fish_eye_y" : "0.0",
    "img_w" : "255",
    "img_h" : "255",
    "img_d" : "3",
    "img_enc" : "JPG",
    "offset_x" : "0.0",
    "offset_y" : "1.0",
    "offset_z" : "2.0",
    "rot_x" : "120.0"
    }'''
        self.send_now(msg)
        time.sleep(0.5)

        msg = '{ "msg_type" : "cam_config_b", "fov" : "150", "fish_eye_x" : "0.0", "fish_eye_y" : "0.0", "img_w" : "255", "img_h" : "255", "img_d" : "3", "img_enc" : "JPG", "offset_x" : "0.0", "offset_y" : "3.0", "offset_z" : "0.0", "rot_x" : "90.0" }'
        self.send_now(msg)
        time.sleep(0.2)

        # Lidar config
        # the offset_x moves camera left/right
        # the offset_y moves camera up/down
        # the offset_z moves camera forward/back
        # degPerSweepInc : as the ray sweeps around, how many degrees does it advance per sample (int)
        # degAngDown : what is the starting angle for the initial sweep compared to the forward vector
        # degAngDelta : what angle change between sweeps
        # numSweepsLevels : how many complete 360 sweeps (int)
        # maxRange : what it max distance we will register a hit
        # noise : what is the scalar on the perlin noise applied to point position
        # Here's some sample settings that similate a more sophisticated lidar:
        # msg = '{ "msg_type" : "lidar_config", "degPerSweepInc" : "2", "degAngDown" : "25", "degAngDelta" : "-1.0", "numSweepsLevels" : "25", "maxRange" : "50.0", "noise" : "0.2", "offset_x" : "0.0", "offset_y" : "1.0", "offset_z" : "1.0", "rot_x" : "0.0" }'
        # And here's some sample settings that similate a simple RpLidar A2 one level horizontal scan.
        msg = '{ "msg_type" : "lidar_config", "degPerSweepInc" : "2", "degAngDown" : "0", "degAngDelta" : "-1.0", "numSweepsLevels" : "1", "maxRange" : "50.0", "noise" : "0.4", "offset_x" : "0.0", "offset_y" : "0.5", "offset_z" : "0.5", "rot_x" : "0.0" }'
        self.send_now(msg)
        time.sleep(0.2)


    def send_controls(self, steering, throttle):
        msg = { "msg_type" : "control",
                "steering" : steering.__str__(),
                "throttle" : throttle.__str__(),
                "brake" : "0.0" }
        self.send(json.dumps(msg))

        #this sleep lets the SDClient thread poll our message and send it out.
        time.sleep(self.poll_socket_sleep_sec)

    def update(self):
        # just random steering now
        st = (self.stearing * 2) - 1
        th = (1 - abs(st)) * 0.3
        self.send_controls(st, th)



In [57]:
from ipywidgets import Layout, Image, HBox
from IPython.display import display

image_widget = Image(format='jpeg', layout=Layout(width='500px', height='500px'))

image_widgetb = Image(format='jpeg', layout=Layout(width='500px', height='500px'))

box = HBox([image_widget, image_widgetb])

In [58]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
display(box)

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(cv2.flip(image,-1))

def update_imageb(change):
    image = change['new']
    image_widgetb.value = bgr8_to_jpeg(cv2.flip(image,-1))
 
logging.basicConfig(level=logging.INFO)

# test params
host = "127.0.0.1" # "trainmydonkey.com" for virtual racing server
port = 9091
num_clients = 1
clients = []
time_to_drive = 120.0


c = SimpleClient((host, port), "Pedro", "Alpine")
c.observe(update_image, names='image')
c.observe(update_imageb, names='imageB')


time.sleep(2)

# Load Scene message. Only one client needs to send the load scene.
msg = '{ "msg_type" : "load_scene", "scene_name" : "mountain_track" }'
c.send_now(msg)


# Send random driving controls
start = time.time()
do_drive = True
while time.time() - start < time_to_drive and do_drive:
    if not c.car_loaded:
        time.sleep(0.3)
        continue
    c.update()
    if c.aborted:
        print("Client socket problem, stopping driving.")
        do_drive = False

time.sleep(3.0)

# Exit Scene - optionally..
msg = '{ "msg_type" : "exit_scene" }'
c.send_now(msg)

# Close down clients
print("waiting for msg loop to stop")
c.stop()

print("clients to stopped")

INFO:gym_donkeycar.core.client:connecting to 127.0.0.1:9091 


{'msg_type': 'scene_selection_ready', 'loaded': '1'}
{'msg_type': 'need_car_config'}
{'msg_type': 'car_loaded'}
{'msg_type': 'cross_start', 'lap_time': '0'}
